# Procedure e processi
Una procedura è uno "schema" di evoluzione *locale* di un processo computazionale. La procedure specifica in ogni fase del processo, come viene eseguito a partire dalla fase precedente. È interessante studiare il comportamento *globale* di un processo la cui evoluzione locale è stata specificata da una procedura. In generale, andare a studiare questo comportamento è molto complicato, ma in alcuni semplici casi si possono identificare degli schemi ricorrenti (in inglese **pattern**).

Proviamo ora a studiare la "forma" che prendono alcuni processi generati da procedure molto semplici. Studieremo anche velocemente la velocità con cui questi processi consumano le due risorse di calcolo fondamentali:

1. IL TEMPO
2. LA MEMORIA

Come nelle lezioni precedenti, le procedure che consideriamo sono molto semplici. Il loro ruolo è lo stesso ricoperto dai *test patterns* in [fotografia](http://www.bealecorner.org/red/test-patterns/): uno schema prototipale ipersemplificato, piuttosto che un caso d'uso reale.

## Ricorsione lineare e iterazione lineare
Si consideri la funzione fattoriale, come definita nell'Esercizio 3.3. Si definisce il fattoriale di un numero naturale $n$, indicato con $n!$, come il prodotto dei numeri naturali uguali o minori a quel numero. In pratica, si definisce:

$$n! = \prod_{k=1}^n k = 1 \cdot 2 \cdot 3 \cdot \cdot \cdot (n-2) \cdot (n-1) \cdot n$$

Ci sono molti modi di calcolare i fattoriali. Un modo è di osservare che $n!$ è uguale ad $n$ volte $(n-1)!$ per qualsiasi intero $n$:

$$ n! = n \cdot [(n-1)\cdot(n-2)\cdots 3 \cdot 2 \cdot 1] = n \cdot (n-1)!$$

In questo modo possiamo calcolare $n!$ calcolando prima $(n-1)!$ e moltiplicando poi il risultato per $n$. Se aggiungiamo la definizione che $1!$ è uguale a 1, possiamo definire la procedura:

In [ ]:
def Fattoriale(n):
    if n == 1:
        return 1
    else:
        return n * Fattoriale(n-1)

In [ ]:
Fattoriale(5)

Se usiamo il *substitution model* per valutare la procedure per $5!$, otteniamo lo schema seguente:

```
Fattoriale(5)
5 * Fattoriale(4)
5 * (4 * Fattoriale(3))
5 * (4 * (3 * Fattoriale(2)))
5 * (4 * (3 * (2 * Fattoriale(1))))
5 * (4 * (3 * (2 * 1)))
5 * (4 * (3 * 2)) 
5 * (4 * 6) 
5 * 24
120
```

Ora, questa procedura è corretta, in quanto produce il risultato richiesto (e si può dimostrare formalmente la correttezza, ma non è lo scopo di questo corso).

Possiamo tuttavia calcolare il fattoriale di un numero usando un approccio diverso. Potremmo descrivere una regola per calcolare il fattoriale di un numero naturale $n$ specificando che prima moltiplichiamo 1 per 2, poi moltiplichiamo il risultato per 3, poi il risultato per 4, e cosi via, sino a raggiungere il valore di $n$. 

Più formalmente, manteniamo una variabile per il prodotto corrente (chiamata in gergo un **accumulator**), insieme a una variabile che "tiene il conto" da 1 a $n$ (un **counter**). Possiamo descrivere il processo di calcolo dicendo che l'accumulator e il counter aggiornano il loro stato "contemporaneamente" ad ogni fase del processo di calcolo:

```
accumulator <- counter * accumulator
counter <- counter + 1
```

Chiaramente, sia l'accumulator che il counter sono inizializzati a 1.

Data questa regola per calcolare $n!$, possiamo scrivere le due procedure seguenti:

In [ ]:
def FactorialIter(accumulator, counter, n):
    if counter > n:
        return accumulator
    else:
        return FactorialIter(counter*accumulator, counter+1, n)
    
def FactorialI(n):
    return FactorialIter(1, 1, n)

In [ ]:
FactorialI(6)

Come prima, se usiamo il *substitution model* possiamo visualizzare il processo di calcolare $5!$ usando la procedura `FactorialI`, come segue:

```
FactorialI(5)
FactorialIter(1, 1, 5)
FactorialIter(1, 2, 5)
FactorialIter(2, 3, 5)
FactorialIter(6, 4, 5)
FactorialIter(24, 5, 5)
FactorialIter(120, 6, 5)
120
```

Si provi a confrontare questo schema di calcolo rispetto al precedente.

A prima vista non sembrano troppo diversi: entrambi calcolano $5!$. Entrambe le procedure richiedono un numero di passi proporzionale al numero $n$. Entrambe le procedure eseguono la stessa sequenza di prodotti, ottenendo gli stessi prodotti parziali. Tuttavia, visualizzando l'evoluzione del processo di calcolo, otteniamo chiaramente una "forma" diversa per i due processi.

Si consideri il primo processo. Procedendo con il modello sostitutivo, otteniamo prima una sequenza di espansioni, seguita da una sequenza di contrazioni. Le espansioni si ottengono mentre il processo costruisce una sequenza di operazioni **deferred** (in questo caso la sequenza di moltiplicazioni). Le contrazioni avvengono quando le operazioni **deferred** sono effettivamente calcolate. 

Questo tipo di processo, caratterizzato da una sequenze di operazioni deffered, viene chiamato **PROCESSO RICORSIVO**. 

Questo processo richiede che l'interprete tenga memoria delle operazioni che deve eseguire dopo, durante la sequenza di contrazioni. Nel caso del calcolo di $n!$, la lunghezza della sequenza di operazioni deferred, e quindi la quantità di informazioni da mantenere in memoria, è lineare nell'input $n$. Si parla quindi di **PROCESSO RICORSIVO LINEARE**.

Al contrario, il secondo processo, quello della procedura `FactorialI(n)` non si espande e non si contrae. Ad ogni passo, gli unici valori di cui dobbiamo tener traccia sono le tre variabili `accumulator`, `counter`, e `n`. Questo viene chiamato un **PROCESSO ITERATIVO**. In generale, un processo iterativo può descritto da un numero finito di *variabili di stato*, insieme con una regola che descrive come le variabili di stato dovrebbero essere aggiornate quando il processo passo da uno stato all'altro, e dovrebbe avere un test d'arresto (i.e., un predicato condizionale `if`) che specifica sotto quali condizioni il processo dovrebbe terminare. Nel calcolare $n!$, il numero di passo richiesto cresce linearmente con $n$. Tale processo viene chiamato un **PROCESSO ITERATIVO LINEARE**.

### IMPORTANTE
Nel confrontare i due tipi di processi, si deve fare attenzione a non confondere il concetto di *processo ricorsivo* con quello di *procedura ricorsiva*. 

Quando descriviamo una procedura come ricorsiva, ci stiamo riferendo al fatto di definire una funzione che si richiama se stessa.

Tuttavia, quando descriviamo un processo che segue uno schema linearmente ricorsivo, stiamo parlando di come il processo evolve, non della sintassi che viene usata per scrivere la procedura. Può creare confusione il fatto che noi diciamo che una procedura ricorsiva, come ad esempio `FactorialI(n)`, genera un processo iterativo. In ogni caso il processo è iterativo: il suo stato è descritto completamente dalle sue tre variabili di stato, e l'interprete deve tener traccia solo di quelle tre variabili per poter eseguire il processo.

Il motivo principale che genera questa confusione, è che l'implementazione attuale di molti di linguaggi di programmazione comuni (C, Java, e lo stesso Python) sono implementati in modo tale che l'esecuzione di ogni chiamata ricorsiva consuma una quantità di memoria che cresce linearmente con il numero di chiamate ricorsive alla stessa procedura, anche se il processo stesso è iterativo.
L'implementazione di altri linguaggi di programmazione (per esempio, il LISP e Haskell) è in grado di distinguere le chiamate ricorsive a processi iterativi, ed è in grado di eseguire tale processo usando una quantità di memoria che dipende solo dal numero di variabili di stato. L'Implementazione di un linguaggio con questa proprietà viene chiamato **tail-recursive**. 

## Ricorsione ad albero
Un altro schema ricorrente di calcolo è la ricorsione ad albero. Si consideri per esempio la successione dei numeri di Fibonacci, in cui ogni numero è la somma dei due numeri precedenti:

$$0, 1, 1, 2, 3, 5, 8, 13, 21, ...$$

In generale, i numeri di Fibonacci possono essere definiti dalla regola seguente:

$$Fib(n) = \left\{ \begin{array}{ll} 
0 & \mbox{if } n = 0 \\  
1 & \mbox{if } n = 1  \\
Fib(n-1) + Fib(n-2) & \mbox{altrimenti} \end{array} \right.$$

**ESERCIZIO 4.1**: Tradurre questa definizione in una procedura ricorsiva per calcolare l'ennesimo numero di Fibonacci.

In [ ]:
# COMPLETARE
# def Fib(n):
#   restituisce l'ennesimo numero della sequenza usando la definizione precedente

Si consideri lo schema di calcolo di tale funzione (FARE ALLA LAVAGNA). Si noti che il processo stesso evolve come un albero: ad ogni nodo il processo di ramifica in due sottoprocessi, tranne che ai nodi foglia. Questo è dovuto al fatto che la procedura chiama se stessa due volte per ogni invocazione.

Questa procedura per calcolare i numeri di Fibonacci è un ottimo esempio di struttura ad albero, ma è un pessimo modo di calcolare i numeri di Fibonacci. Per rendersi conto di quanto sia inefficiente, si consideri che questo processo usa un numero di passi che cresce esponenzialmente con l'input. Tuttavia, lo spazio richiesto cresce solo linearmente con l'input, in quanto dobbiamo tener traccia dei nodi al di sotto del nodo corrente a qualsiasi momento del processo. 
In generale, il numero di passi richiesto da un processo ricorsivo ad albero sarà proporzionale al numero di nodi dell'albero, mentre lo spazio richiesto sarà proporzionale alla profondità massima dell'albero.

Il calcolo dei numeri di Fibonacci può essere formulato anche come un processo iterativo. L'idea è di usare una coppia di numeri $a$ e $b$, inizializzati con $Fib(1)=1$ e $Fib(0)=0$, e di applicare ripetutamente le trasformazioni simultanee:

```
a <- a + b
b <- a
```

Non dovrebbe essere complicato realizzare che dopo aver applicato queste trasformazioni $n$ volte, $a$ e $b$ avranno i valori $Fib(n+1)$ e $Fib(n)$.

**ESERCIZIO 4.2**: Scrivere una procedura che calcoli i numeri di Fibonacci usando un processo iterativo.


## Calcolo di elevamento a potenza
Si consideri il problema di calcolare l'esponenziale di un numero dato. Vorremo avere una procedura che prende come argomenti la base $b$ e un intero positivo $n$, e calcola il valore $b^n$. Un modo possibile di procedere è utilizzando la definizione ricorsiva:

```
b^n = b \cdot b^{n-1}
b^0 = 1
```

**ESERCIZIO 4.3**: Scrivere una procedura che applichi direttamente la definizione precedente, tramite un processo ricorsivo lineare.

In [ ]:
# COMPLETARE

**ESERCIZIO 4.4**: Scrivere una procedura che esegue l'elevamento a potenza usando un processo iterativo lineare.

In [ ]:
# COMPLETARE

### Massimo Comun Divisore
Il Massimo Comun Divisore (MCD) di due numeri intero $a$ e $b$ è definito come il più grande numero intero che divide sia $a$ che $b$ senza resto. Per esempio, il massimo comun divisore di 16 e 28 è il numero 4. Esiste un metodo famoso per calcolare tale numero: l'**algoritmo di Euclide**.

L'idea dell'algoritmo si basa sull'osservazione che, se $r$ è il resto di quando $a$ è diviso per $b$, allora i divisori comuni di $a$ e $b$ sono esattamente esattamente gli stessi divisori comuni tra $b$ e $r$. Quindi possiamo usare l'equazione:

$$MCD(a,b) = MCD(n,r)$$

per ridurre il problema di trovare i divisori comuni calcolando il MCD tra coppie di numeri interi via via più piccoli. Per esempio:

```
MCD(206, 4) = MCD(40, 6)
            = MCD(6,4)
            = MCD(4,2)
            = MCD(2,0)
            = 2
```

**ESERCIZIO 4.3**: Scrivere una procedura che calcola il massimo comun divisore usando l'algoritmo di Euclide (per calcolare il resto di una divisione tra due numeri interi si usa l'operatore modulo `%`).

In [ ]:
7%4